In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.models.DCVC_net import DCVC_net
import torch
from torchvision import transforms
import numpy as np
import pathlib
import os
import matplotlib.pyplot as plt
import wandb
import tqdm
from torchnet import meter

/home/jatin/miniconda3/envs/DCVC/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
BATCH_SIZE = 4
DATA_DIRS = [
    pathlib.Path('../uvg/Beauty_PNG_1024/'),
    pathlib.Path('../uvg/Bosphorus_PNG_1024/'),
    pathlib.Path('../uvg/HoneyBee_PNG_1024/'),
    pathlib.Path('../uvg/Jockey_PNG_1024/'),
    pathlib.Path('../uvg/ReadySetGo_PNG_1024/'),
    pathlib.Path('../uvg/ShakeNDry_PNG_1024/'),
    pathlib.Path('../uvg/YachtRide_PNG_1024/'),
]
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

In [5]:
video_net = DCVC_net()

In [6]:
chpt = torch.load('checkpoints/model_dcvc_quality_3_psnr.pth')
# chpt = torch.load('reproduce-dcvc-lamb-2048-ft/dcvc_epoch=2_psnr=39.108_bpp=0.0807.pt')
# video_net.load_state_dict(chpt['model'], strict=True)
video_net.load_state_dict(chpt, strict=True)

# LOADS OLD WEIGHTS
# chpt = torch.load('dcvc-b-frame-convtranspose-fail/dcvc_epoch=4_int.pt')
# temporalPriorEncoder = video_net.temporalPriorEncoder
# del video_net.temporalPriorEncoder
# video_net.load_state_dict(chpt['model'], strict=False)
# video_net.temporalPriorEncoder = temporalPriorEncoder

video_net = video_net.to(DEVICE)
del chpt

In [7]:
class UVGDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, crop_size=None, deterministic=False):
        self.data_dir = data_dir
        self.crop_size = crop_size
        self.deterministic = deterministic
        self.numframes = len(os.listdir(data_dir))
        if crop_size is not None:
            self.transforms = torch.nn.Sequential(
                transforms.CenterCrop(crop_size)
            )
        else:
            self.transforms = None
       
    def __getitem__(self, i):
        interval = 1
        # this is 1-indexed on disk
        i += 1
        int_as_str = f'{i}'.zfill(5)
        ref = plt.imread(self.data_dir / f'im{int_as_str}.png')
        int_as_str = f'{i + interval}'.zfill(5)
        im = plt.imread(self.data_dir / f'im{int_as_str}.png')
        imgs = [ref, im]

        # plt.imread should make inputs in [0, 1] for us
        imgs = np.stack(imgs, axis=0)
        # bring RGB channels in front
        imgs = imgs.transpose(0, 3, 1, 2)
        imgs = torch.FloatTensor(imgs)
        if self.transforms:
            imgs = self.transforms(imgs)
        return imgs

    def __len__(self):
        return self.numframes - 1

all_uvg_dl = []
for d in DATA_DIRS:
    ds = UVGDataset(d, crop_size=640)
    dl = torch.utils.data.DataLoader(
        ds,
        shuffle=True,
        batch_size=BATCH_SIZE,
        num_workers=6,
        prefetch_factor=5
    )
    all_uvg_dl.append(dl)


In [8]:
def count_parameters(model):
    """Return number of parameters in a model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(video_net)} trainable parameters')

The model has 7944448 trainable parameters


In [9]:
def test_epoch(model, dl, compress_type, max_iters=None):
    mse_criterion = torch.nn.MSELoss()
    model.train()
    
    mse_meter = meter.AverageValueMeter()
    bpp_meter = meter.AverageValueMeter()
    bpp_mv_y_meter = meter.AverageValueMeter()
    bpp_mv_z_meter = meter.AverageValueMeter()
    bpp_y_meter = meter.AverageValueMeter()
    bpp_z_meter = meter.AverageValueMeter()
    
    pbar = tqdm.tqdm(dl)
    for i, x in enumerate(pbar):
        x = x.to(DEVICE)
        ref = x[:,0]
        im = x[:,1]
        with torch.no_grad():
            preds_dict = model(ref, im, compress_type=compress_type)
        preds = preds_dict['recon_image']
        bpp = preds_dict['bpp']
        
        mse_ls = mse_criterion(preds, im)
        mse_meter.add(mse_ls.item())
        bpp_meter.add(bpp.item())
        bpp_mv_y_meter.add(preds_dict['bpp_mv_y'].item())
        bpp_mv_z_meter.add(preds_dict['bpp_mv_z'].item())
        bpp_y_meter.add(preds_dict['bpp_y'].item())
        bpp_z_meter.add(preds_dict['bpp_z'].item())
        if i % 1 == 0:
            avg_psnr = round(-10.0*np.log10(mse_meter.value()[0]), 6)
            avg_bpp = round(bpp_meter.value()[0], 6)
            avg_bpp_mv_y = round(bpp_mv_y_meter.value()[0], 6)
            avg_bpp_mv_z = round(bpp_mv_z_meter.value()[0], 6)
            avg_bpp_y = round(bpp_y_meter.value()[0], 6)
            avg_bpp_z = round(bpp_z_meter.value()[0], 6)
            msg = (
                f'Avg PSNR: {avg_psnr}, bpp: {avg_bpp}, bpp_mv_y: {avg_bpp_mv_y}, avg_bpp_mv_z: {avg_bpp_mv_z} '
                f'avg_bpp_y: {avg_bpp_y}, avg_bpp_z: {avg_bpp_z}'
            )
            pbar.set_description(msg)
            
        if max_iters is not None and i == max_iters:
            break
    
    avg_psnr = -10.0*np.log10(mse_meter.value()[0])
    avg_bpp = bpp_meter.value()[0]                     
    return avg_psnr, avg_bpp
            

In [10]:
# US
psnrs = []
bpps = []
for p, dl in zip(DATA_DIRS, all_uvg_dl):
    print("RUNNING:", p)
    psnr, bpp = test_epoch(video_net, dl, compress_type='full', max_iters=10)
    psnrs.append(psnr)
    bpps.append(bpp)

RUNNING: ../uvg/Beauty_PNG_1024


Avg PSNR: 35.792831, bpp: 0.06993, bpp_mv_y: 0.026319, avg_bpp_mv_z: 0.000757 avg_bpp_y: 0.041638, avg_bpp_z: 0.001215:   3%|▎         | 5/150 [00:04<02:15,  1.07it/s] 


KeyboardInterrupt: 

In [27]:
np.mean(psnrs), np.mean(bpps)

(36.76519039425703, 0.09219490400479212)

In [11]:
psnrs = []
bpps = []
for p, dl in zip(DATA_DIRS, all_uvg_dl):
    print("RUNNING:", p)
    psnr, bpp = test_epoch(video_net, dl, compress_type='full', max_iters=10)
    psnrs.append(psnr)
    bpps.append(bpp)

RUNNING: ../uvg/Beauty_PNG_1024


Avg PSNR: 35.808189, bpp: 0.071011, bpp_mv_y: 0.027134, avg_bpp_mv_z: 0.000756 avg_bpp_y: 0.041912, avg_bpp_z: 0.001209:   7%|▋         | 10/150 [00:07<01:45,  1.32it/s]


RUNNING: ../uvg/Bosphorus_PNG_1024


Avg PSNR: 41.205318, bpp: 0.024956, bpp_mv_y: 0.009829, avg_bpp_mv_z: 0.000772 avg_bpp_y: 0.013315, avg_bpp_z: 0.001041:   7%|▋         | 10/150 [00:07<01:46,  1.31it/s]


RUNNING: ../uvg/HoneyBee_PNG_1024


...

Avg PSNR: 40.021042, bpp: 0.07852, bpp_mv_y: 0.020447, avg_bpp_mv_z: 0.001086 avg_bpp_y: 0.055624, avg_bpp_z: 0.001363:   7%|▋         | 10/150 [00:07<01:46,  1.31it/s]


RUNNING: ../uvg/ShakeNDry_PNG_1024


Avg PSNR: 37.064192, bpp: 0.194562, bpp_mv_y: 0.039337, avg_bpp_mv_z: 0.001045 avg_bpp_y: 0.152696, avg_bpp_z: 0.001484:  13%|█▎        | 10/75 [00:07<00:50,  1.28it/s]


RUNNING: ../uvg/YachtRide_PNG_1024


Avg PSNR: 40.224215, bpp: 0.104782, bpp_mv_y: 0.02252, avg_bpp_mv_z: 0.000996 avg_bpp_y: 0.079532, avg_bpp_z: 0.001734:   7%|▋         | 10/150 [00:07<01:50,  1.27it/s]


In [12]:
np.mean(psnrs), np.mean(bpps)

(38.6784086338597, 0.08021740375884942)